# Data Import

In [339]:
UDS_TRAIN['DECSUB.1']

85701    1
12583    1
96670    0
96103    1
4948     1
        ..
74255    0
39954    1
36744    0
85819    1
91693    0
Name: DECSUB.1, Length: 90465, dtype: int64

In [346]:
UDS_TRAIN['DECSUB'] == UDS_TRAIN['DECSUB.1']

85701    True
12583    True
96670    True
96103    True
4948     True
         ... 
74255    True
39954    True
36744    True
85819    True
91693    True
Length: 90465, dtype: bool

In [248]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
matplotlib.style.use('fivethirtyeight')
# Edit this part

In [249]:
# # Import PyDrive and associated libraries.
# # This only needs to be done once per notebook.
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client.
# # This only needs to be done once per notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# # Download a file based on its file ID

# files = {
#     # id : name
#     "1DdooWeRf-z96yLWRQJgZKNQySEwK2cFm": "CSF_TEST",
#     "1ENlaC2kmG9IM8AM7ILN3yjATVDurb0_d": "CSF_TRAIN",
#     "1gKoMeOwwAAW5D0rW-6REPgNVilo8skQ2": "MRI_TEST",
#     "17LNQwMOI305oStnWiXYTevjo0rGvMpnD": "MRI_TRAIN",
#     "1WqTBJuw1uAsQdtseT8fhc6y-9C0uXsqa": "UDS_TEST",
#     "17aHVkjjot2UehCu4weRawCCpJ5IYFIFt": "UDS_TRAIN",
#     "1TCJLLMLMXGgf3G8RFmmt097STFVDZvKU": "UDS_MRI_CSF_TEST",
#     "1MNpjLPC0AfZLJu4eUE0iS_3Su2eAn8JO": "UDS_MRI_CSF_TRAIN",
#     "1PDwCyerMGYdm-QmtqNj-fyv_Jx7rEXwO": "UDS_MRI_TRAIN",
#     "1-RgmD66M1Opxe0UvGkiMeW97QvdsMFet": "UDS_MRI_TEST"
# }

# for key, value in files.items():
#     downloaded = drive.CreateFile({'id': key})
#     downloaded.GetContentFile(value + ".csv")
#     _ = pd.read_csv(value + ".csv", index_col="Unnamed: 0")

#     # PUT NACCUDSD AT THE END!
#     columns = list(_.columns)
#     columns.remove("NACCUDSD")
#     columns.append("NACCUDSD")

#     _ = _.reindex(columns=columns)

#     globals()[value] = _

# # This code will make bunch variable with the names above. like MRI_TEST will be a variable with MRI_TEST.csv from google drive as a dataframe.

In [250]:
# FOR LOCAL USE


# FOR LOCAL USE
names = ["CSF_TEST",
"CSF_TRAIN",
"MRI_TEST",
"MRI_TRAIN",
"UDS_TEST",
"UDS_TRAIN",
"UDS_MRI_CSF_TEST",
"UDS_MRI_CSF_TRAIN",
"UDS_MRI_TRAIN",
"UDS_MRI_TEST"]

for name in names:
    _ = pd.read_csv("C:/Users/tminh/Downloads/results of yueqi code/DATA/Final 6.27 - " + name + ".csv", index_col="Unnamed: 0")

    # PUT NACCUDSD AT THE END!
    columns = list(_.columns)
    columns.remove("NACCUDSD")
    columns.append("NACCUDSD")

    _ = _.reindex(columns=columns)

    globals()[name] = _


In [251]:
def binarize(df):
  new_df = df.copy()
  new_df['NACCUDSD'] = new_df['NACCUDSD'].replace(1, 0).replace([2, 3, 4], 1)
  return new_df

# Fusion Methods

## AE

In [252]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import gc

def AE(df, epochs = 10):

  # Separate the input features (explanatory variables) and the response variable
  X = df.drop('NACCUDSD', axis=1)  # Input features
  y = df['NACCUDSD']  # Response variable

  # Perform one-hot encoding on the categorical variable
  encoder = LabelEncoder()
  y_encoded = encoder.fit_transform(y)
  y_categorical = pd.get_dummies(y_encoded)

  # # Split the data into training and testing sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0., random_state=42)
  y_train = y_categorical

  # Scale the input features
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X)
  # X_test_scaled = scaler.transform(X_test)

  # Create the neural network model
  input_shape=(X_train_scaled.shape[1],)
  print(input_shape)

  model = Sequential()
  model.add(Dense(64, activation='relu', input_shape = input_shape))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(4, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(input_shape[0], activation='softmax'))  # Output layer with softmax activation for categorical variables

  # Compile the model
  model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

  # Train the model
  model.fit(X_train_scaled, X_train_scaled, epochs=epochs, batch_size=16)

  return model, scaler
# PART 2: GET VALUES

def through_AE(df, model, scaler):
  print("started")

  # Separate the input features (explanatory variables) and the response variable
  X = df.drop('NACCUDSD', axis=1)  # Input features
  y = df['NACCUDSD']

  # Perform one-hot encoding on the categorical variable
  encoder = LabelEncoder()
  y_encoded = encoder.fit_transform(y)
  y_categorical = pd.get_dummies(y_encoded)

  # Scale the input features
  # scaler = StandardScaler()
  X_scaled = scaler.transform(X)


  import tensorflow as tf
  # Create a new model to get values from the last dense layer before output
  last_dense_layer_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-4].output)

  # table = []
  # for i in range(0, len(X_scaled)): #
  #   # print(i)
  #   if i % 100 == 0:
  #     print(str(i) + " / " + str(len(X_scaled)))

  #   input_row = X_scaled[i]  # Replace with your desired input row

  #   # Obtain values from the last dense layer before output for the input row
  #   output_values = last_dense_layer_model.predict(input_row.reshape(1, -1), verbose=0)[0]

  #   table.append(output_values)
  #   gc.collect()

  # print('Output values:', output_values)

  output_values = last_dense_layer_model.predict(X_scaled)
  new_df = pd.DataFrame(output_values)
  new_df['NACCUDSD'] = df.reset_index()['NACCUDSD']

  return new_df

## LASSO

In [253]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

def lasso(df, al, thresh):

  # Create the Lasso model
  lasso = Lasso(alpha=al)

  # Select the features
  selector = SelectFromModel(lasso, threshold=thresh)
  selector.fit(df.drop("NACCUDSD", axis=1), df["NACCUDSD"])

  # Get the selected features
  selected_features = selector.get_support()

  return df.columns[:-1][selected_features].tolist()

## SDNN



In [254]:
def through_SDNN_old(df, model, scaler):

  # Separate the input features (explanatory variables) and the response variable
  X = df.drop('NACCUDSD', axis=1)  # Input features
  y = df['NACCUDSD']

  # Perform one-hot encoding on the categorical variable
  encoder = LabelEncoder()
  y_encoded = encoder.fit_transform(y)
  y_categorical = pd.get_dummies(y_encoded)

  # # Split the data into training and testing sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

  # Scale the input features
  # scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)

  import tensorflow as tf
  # Create a new model to get values from the last dense layer before output
  last_dense_layer_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)


    
  pickle = []
  for i in range(0, len(X_scaled)):
    # print(i)
    if i % 1000 == 0:
      print(str(i) + " / " + str(len(X_scaled)))
      that_df = pd.DataFrame(pickle)

    input_row = X_scaled[i]  # Replace with your desired input row

    # Obtain values from the last dense layer before output for the input row
    output_values = last_dense_layer_model.predict(input_row.reshape(1, -1), verbose=0)[0]
    # print(output_values)
    pickle.append(output_values)
    gc.collect()



  # print('Output values:', output_values)
  # new_df = pd.DataFrame(output_values)
  new_df = pd.DataFrame(pickle)
  new_df['NACCUDSD'] = df.reset_index()['NACCUDSD']

  return new_df

In [255]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import gc

def SDNN(df, epochs = 10):

  # Separate the input features (explanatory variables) and the response variable
  X = df.drop('NACCUDSD', axis=1)  # Input features
  y = df['NACCUDSD']  # Response variable

  # Perform one-hot encoding on the categorical variable
  encoder = LabelEncoder()
  y_encoded = encoder.fit_transform(y)
  y_categorical = pd.get_dummies(y_encoded)

  # # Split the data into training and testing sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0., random_state=42)
  y_train = y_categorical

  # Scale the input features
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X)
  # X_test_scaled = scaler.transform(X_test)

  # Create the neural network model
  model = Sequential()
  model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(y_categorical.shape[1], activation='softmax'))  # Output layer with softmax activation for categorical variables

  # Compile the model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  # Train the model
  model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=16)

  return model, scaler
# PART 2: GET VALUES

def through_SDNN(df, model, scaler):

  # Separate the input features (explanatory variables) and the response variable
  X = df.drop('NACCUDSD', axis=1)  # Input features
  y = df['NACCUDSD']

  # Perform one-hot encoding on the categorical variable
  encoder = LabelEncoder()
  y_encoded = encoder.fit_transform(y)
  y_categorical = pd.get_dummies(y_encoded)

  # # Split the data into training and testing sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

  # Scale the input features
  # scaler = StandardScaler()
  X_scaled = scaler.transform(X)

    
  import tensorflow as tf
  # Create a new model to get values from the last dense layer before output
  last_dense_layer_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)

  # pickle = []
  # for i in range(0, len(X_scaled)): #
  #   # print(i)
  #   if i % 1000 == 0:
  #     print(str(i) + " / " + str(len(X_scaled)))
  #     that_df = pd.DataFrame(pickle)

  #   input_row = X_scaled[i]  # Replace with your desired input row

  #   # Obtain values from the last dense layer before output for the input row
  #   output_values = last_dense_layer_model.predict(input_row.reshape(1, -1), verbose=0)[0]
  #   print(output_values)
  #   pickle.append(output_values)
  #   gc.collect()

  output_values = last_dense_layer_model.predict(X_scaled, verbose=0)

  # print('Output values:', output_values)
  new_df = pd.DataFrame(output_values)
  # new_df = pd.DataFrame(pickle)
  new_df['NACCUDSD'] = df.reset_index()['NACCUDSD']

  return new_df

## PCA

In [256]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def pca(train_df, test_df, var = 0.5):


  # Load and preprocess the training dataset
  train_data = train_df.reset_index().copy()
  train_features = train_data.drop('NACCUDSD', axis=1)  # Exclude the response variable

  # Standardize the training features by removing the mean and scaling to unit variance
  scaler = StandardScaler()
  scaled_train_features = scaler.fit_transform(train_features)

  # Perform PCA on the training features
  pca = PCA()
  pca.fit(scaled_train_features)

  # Load and preprocess the test dataset
  test_data = test_df.reset_index().copy()
  test_features = test_data.drop('NACCUDSD', axis=1)  # Exclude the response variable

  # Standardize the test features using the same scaler applied to the training features
  scaled_test_features = scaler.transform(test_features)

  # Apply the learned PCA model to both training and test features
  train_pca = pca.transform(scaled_train_features)
  test_pca = pca.transform(scaled_test_features)

  # Construct new DataFrames for the transformed features
  train_pca_df = pd.DataFrame(data=train_pca, columns=[f'PC{i+1}' for i in range(pca.n_components_)])
  test_pca_df = pd.DataFrame(data=test_pca, columns=[f'PC{i+1}' for i in range(pca.n_components_)])

  # Concatenate the response variable with the PCA-transformed training features
  train_pca_df = pd.concat([train_pca_df, train_data['NACCUDSD']], axis=1)
  test_pca_df = pd.concat([test_pca_df, test_data['NACCUDSD']], axis=1)

  # Print the resulting PCA-transformed datasets
  return [train_pca_df, test_pca_df]


# Classification Methods

## LogReg

In [257]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def logistic_regression(train_df, test_df, cols):

  train_df = binarize(train_df)
  test_df =  binarize(test_df)


  X_train = train_df[cols]  # Input features # ERROR: y is included
  y_train = train_df['NACCUDSD']   # Target variable

  X_test = test_df[cols]  # Input features # ERROR: y is included
  y_test = test_df['NACCUDSD']   # Target variable

  logreg = LogisticRegression(solver='lbfgs')
  logreg.fit(X_train, y_train)

  y_pred = logreg.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  return accuracy

## RANDOM FOREST

In [258]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


def random_forest(train_df, test_df, cols):
  # Load the dataset
  # dataset = UDS_MRI_CSF[concat_cols+['NACCUDSD']]
  X_train = train_df[cols]  # Input features # ERROR: y is included
  y_train = train_df['NACCUDSD']   # Target variable

  X_test = test_df[cols]  # Input features # ERROR: y is included
  y_test = test_df['NACCUDSD']   # Target variable

  # Split the data into training and testing sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


  rf_classifier = RandomForestClassifier(n_estimators=50)

  # Fit the model to the training data
  rf_classifier.fit(X_train, y_train)

  # Make predictions on the test set
  y_pred = rf_classifier.predict(X_test)

  # Evaluate model performance
  accuracy = accuracy_score(y_test, y_pred)
  # print("Accuracy:", accuracy)
  return accuracy


## Feed forward neural network

In [259]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# Assuming you have train_df and test_df DataFrames
# and the response variable is 'NACCUDSD'

def neural_network(train_df, test_df, cols, epochs = 10):
  # Separate features and target variable
  X_train = train_df[cols]
  y_train = train_df['NACCUDSD']-1
  X_test = test_df[cols]
  y_test = test_df['NACCUDSD']-1

  # Convert the target variable to categorical
  num_classes = len(set(y_train))
  y_train_cat = to_categorical(y_train, num_classes)
  y_test_cat = to_categorical(y_test, num_classes)

  # Standardize the features
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)

  # Create the model
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))

  # Compile the model
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # Train the model
  model.fit(X_train_scaled, y_train_cat, epochs=epochs, batch_size=32, verbose=1)

  # Evaluate the model
  loss, accuracy = model.evaluate(X_test_scaled, y_test_cat, verbose=0)

  print("Test loss:", loss)
  print("Test accuracy:", accuracy)

  return model

## Column Names

In [260]:
input_mri_feats = ['NACCICV', 'NACCBRNV',
       'NACCWMVL', 'CSFVOL', 'GRAYVOL', 'WHITEVOL', 'WMHVOL', 'HIPPOVOL',
       'CEREALL', 'CERETISS', 'CERECSF', 'CEREGR', 'CEREWH', 'LHIPPO',
       'RHIPPO', 'LLATVENT', 'RLATVENT', 'LATVENT', 'THIRVENT', 'LFRCORT',
       'RFRCORT', 'FRCORT', 'LOCCORT', 'ROCCORT', 'OCCCORT', 'LPARCORT',
       'RPARCORT', 'PARCORT', 'LTEMPCOR', 'RTEMPCOR', 'TEMPCOR', 'LCAC',
       'LCACM', 'LCMF', 'LCMFM', 'LCUN', 'LCUNM', 'LENT', 'LENTM', 'LFUS',
       'LFUSM', 'LINFPAR', 'LINFPARM', 'LINFTEMP', 'LINFTEMM', 'LINSULA',
       'LINSULAM', 'LISTHC', 'LISTHCM', 'LLATOCC', 'LLATOCCM', 'LLATORBF',
       'LLATORBM', 'LLING', 'LLINGM', 'LMEDORBF', 'LMEDORBM', 'LMIDTEMP',
       'LMIDTEMM', 'LPARCEN', 'LPARCENM', 'LPARHIP', 'LPARHIPM',
       'LPARSOP', 'LPARSOPM', 'LPARORB', 'LPARORBM', 'LPARTRI',
       'LPARTRIM', 'LPERCAL', 'LPERCALM', 'LPOSCEN', 'LPOSCENM',
       'LPOSCIN', 'LPOSCINM', 'LPRECEN', 'LPRECENM', 'LPRECUN',
       'LPRECUNM', 'LROSANC', 'LROSANCM', 'LROSMF', 'LROSMFM', 'LSUPFR',
       'LSUPFRM', 'LSUPPAR', 'LSUPPARM', 'LSUPTEM', 'LSUPTEMM', 'LSUPMAR',
       'LSUPMARM', 'LTRTEM', 'LTRTEMM', 'RCAC', 'RCACM', 'RCMF', 'RCMFM',
       'RCUN', 'RCUNM', 'RENT', 'RENTM', 'RFUS', 'RFUSM', 'RINFPAR',
       'RINFPARM', 'RINFTEMP', 'RINFTEMM', 'RINSULA', 'RINSULAM',
       'RISTHC', 'RISTHCM', 'RLATOCC', 'RLATOCCM', 'RLATORBF', 'RLATORBM',
       'RLING', 'RLINGM', 'RMEDORBF', 'RMEDORBM', 'RMIDTEMP', 'RMIDTEMM',
       'RPARCEN', 'RPARCENM', 'RPARHIP', 'RPARHIPM', 'RPARSOP',
       'RPARSOPM', 'RPARORB', 'RPARORBM', 'RPARTRI', 'RPARTRIM',
       'RPERCAL', 'RPERCALM', 'RPOSCEN', 'RPOSCENM', 'RPOSCIN',
       'RPOSCINM', 'RPRECEN', 'RPRECENM', 'RPRECUN', 'RPRECUNM',
       'RROSANC', 'RROSANCM', 'RROSMF', 'RROSMFM', 'RSUPFR', 'RSUPFRM',
       'RSUPPAR', 'RSUPPARM', 'RSUPTEM', 'RSUPTEMM', 'RSUPMAR',
       'RSUPMARM', 'RTRTEM', 'RTRTEMM']

input_demo_feats = ['SEX', 'HISPANIC', 'HISPOR', 'RACE', 'RACEX',
       'PRIMLANG', 'EDUC', 'MARISTAT', 'NACCLIVS', 'INDEPEND', 'RESIDENC',
       'NACCNIHR','NACCAGE']
input_copart_feats = ['INEDUC', 'INRELTO', 'INKNOWN', 'INLIVWTH', 'INVISITS', 'INCALLS',
       'INRELY']
input_fam_hist_feats = ['NACCFAM', 'NACCMOM', 'NACCDAD', 'NACCAM', 'NACCAMX',
       'NACCAMS', 'NACCAMSX', 'NACCFM', 'NACCFMX', 'NACCFMS', 'NACCFMSX',
       'NACCOM', 'NACCOMX', 'NACCOMS', 'NACCOMSX', 'NACCFADM', 'NACCFFTD']
input_patient_hist_feats = ['ANYMEDS', 'TOBAC30', 'TOBAC100', 'SMOKYRS', 'PACKSPER',
       'QUITSMOK', 'CVHATT', 'CVAFIB', 'CVANGIO', 'CVBYPASS', 'CVPACDEF',
       'CVPACE', 'CVCHF', 'CVOTHR', 'CBSTROKE', 'NACCSTYR', 'CBTIA',
       'NACCTIYR', 'SEIZURES', 'NACCTBI', 'TBI', 'TBIBRIEF', 'TRAUMBRF',
       'TBIEXTEN', 'TRAUMEXT', 'TBIWOLOS', 'TRAUMCHR', 'TBIYEAR',
       'NCOTHR', 'DIABETES', 'DIABTYPE', 'HYPERTEN', 'HYPERCHO', 'B12DEF',
       'THYROID', 'ARTHRIT', 'ARTHTYPE', 'ARTHTYPX', 'ARTHUPEX',
       'ARTHLOEX', 'ARTHSPIN', 'ARTHUNK', 'INCONTU', 'INCONTF', 'APNEA',
       'RBD', 'INSOMN', 'OTHSLEEP', 'OTHSLEEX', 'ALCOHOL', 'ABUSOTHR',
       'ABUSX', 'PTSD', 'BIPOLAR', 'SCHIZ', 'DEP2YRS', 'DEPOTHR',
       'ANXIETY', 'OCD', 'NPSYDEV', 'PSYCDIS', 'PSYCDISX',
       'NACCAAAS', 'NACCAANX', 'NACCAC', 'NACCACEI',
       'NACCADEP', 'NACCAHTN', 'NACCAMD', 'NACCANGI', 'NACCAPSY',
       'NACCBETA', 'NACCCCBS', 'NACCDBMD', 'NACCDIUR', 'NACCEMD',
       'NACCEPMD', 'NACCHTNC', 'NACCLIPL', 'NACCNSD', 'NACCPDMD',
       'NACCVASD']
input_physical_feats = ['HEIGHT','WEIGHT', 'BPSYS', 'BPDIAS', 'HRATE',
        'VISION', 'VISCORR','VISWCORR', 'HEARING', 'HEARAID', 'HEARWAID',
        'NACCBMI']
input_exam_feats = ['FOCLSYM','FOCLSIGN','NACCNREX', 'NORMEXAM', 'DECSUB']
input_npi_feats = ['NPIQINF', 'NPIQINFX', 'DEL', 'DELSEV', 'HALL',
       'HALLSEV', 'AGIT', 'AGITSEV', 'DEPD', 'DEPDSEV', 'ANX', 'ANXSEV',
       'ELAT', 'ELATSEV', 'APA', 'APASEV', 'DISN', 'DISNSEV', 'IRR',
       'IRRSEV', 'MOT', 'MOTSEV', 'NITE', 'NITESEV', 'APP', 'APPSEV']
input_gds_feats = ['NOGDS', 'SATIS', 'DROPACT', 'EMPTY', 'BORED', 'SPIRITS', 'AFRAID',
       'HAPPY', 'HELPLESS', 'STAYHOME', 'MEMPROB', 'WONDRFUL', 'WRTHLESS',
       'ENERGY', 'HOPELESS', 'BETTER', 'NACCGDS']
input_faq_feats = ['BILLS', 'TAXES','SHOPPING', 'GAMES', 'STOVE',
        'MEALPREP', 'EVENTS', 'PAYATTN','REMDATES', 'TRAVEL']
input_scd_feats = ['DECSUB']
intput_np_feats = ['MMSEORDA',
       'MMSEORLO', 'PENTAGON', 'NACCMMSE', 'LOGIMEM', 'MEMUNITS',
       'MEMTIME', 'DIGIF', 'DIGIFLEN', 'DIGIB', 'DIGIBLEN', 'ANIMALS',
       'VEG', 'TRAILA', 'TRAILARR', 'TRAILALI', 'TRAILB', 'TRAILBRR',
       'TRAILBLI', 'BOSTON', 'MOCATOTS']
input_gene_feats = ['NACCAPOE']
label_feat = ['NACCUDSD']
# cdr_feats = ['MEMORY', 'ORIENT', 'JUDGMENT', 'COMMUN', 'HOMEHOBB', 'PERSCARE', 'CDRSUM', 'CDRGLOB']
UDS_selected_features = label_feat + input_gene_feats + intput_np_feats + input_scd_feats + input_faq_feats + input_gds_feats + input_npi_feats + input_exam_feats + input_physical_feats + input_patient_hist_feats + input_fam_hist_feats + input_copart_feats

# UDS

## PCA + three

In [261]:
pca_train, pca_test = pca(train_df = UDS_TRAIN, test_df = UDS_TEST)

In [262]:
random_forest(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])


KeyboardInterrupt



In [ ]:
logistic_regression(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])

In [ ]:
neural_network(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])

## Lasso + random forest & logistic regression & neural network

In [ ]:
cols = lasso(df = UDS_TRAIN, al = 0.01, thresh = 0.04)
cols

In [ ]:
random_forest(train_df = UDS_TRAIN, test_df = UDS_TEST, cols = cols)

In [ ]:
logistic_regression(train_df = UDS_TRAIN, test_df = UDS_TEST, cols = cols)

In [ ]:
neural_network(train_df = UDS_TRAIN, test_df = UDS_TEST, cols = cols)

## SDNN + random forest

In [ ]:
mod,scaler = SDNN(UDS_TRAIN)

In [ ]:
processed_df_train = through_SDNN(UDS_TRAIN, mod, scaler)
processed_df_test = through_SDNN(UDS_TEST, mod, scaler)

In [ ]:
logistic_regression(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])

In [ ]:
random_forest(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])

In [ ]:
neural_network(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])

## AE + the three

In [ ]:
model, scaler = AE(UDS_TRAIN, epochs = 50)

In [ ]:
processed_AE_train = through_AE(UDS_TRAIN, model, scaler)

In [ ]:
# processed_AE_train.to_csv('uds_AE_train_proc.csv')
processed_AE_test = through_AE(UDS_TEST, model, scaler)

In [ ]:
logistic_regression(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])

In [ ]:
neural_network(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])

In [ ]:
random_forest(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])

# MRI

### AE + three

In [ ]:
model,scaler = AE(MRI_TRAIN, epochs = 50)

In [ ]:
processed_AE_train = through_AE(MRI_TRAIN, model, scaler)
processed_AE_test = through_AE(MRI_TEST, model, scaler)

In [ ]:
random_forest(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])

In [ ]:
logistic_regression(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])

In [ ]:
neural_network(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])

### PCA + three

In [ ]:
pca_train, pca_test = pca(train_df = MRI_TRAIN, test_df = MRI_TEST, var = 0.3)

In [ ]:
random_forest(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])

In [ ]:
logistic_regression(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])

In [ ]:
neural_network(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])

### Lasso + random forest & logistic regression & neural network

In [ ]:
cols = lasso(df = MRI_TRAIN, al = 0.01, thresh = 0.05)
cols

In [ ]:
random_forest(train_df = MRI_TRAIN, test_df = MRI_TEST, cols = cols)

In [ ]:
logistic_regression(train_df = MRI_TRAIN, test_df = MRI_TEST, cols = cols)

In [ ]:
neural_network(train_df = MRI_TRAIN, test_df = MRI_TEST, cols = cols)

### SDNN + random forest & logistic regression & neural network

In [ ]:
mod, scaler = SDNN(MRI_TRAIN)

In [ ]:
processed_df_train = through_SDNN(MRI_TRAIN, mod, scaler)
processed_df_test = through_SDNN(MRI_TEST, mod, scaler)

In [ ]:
processed_df_train.columns = processed_df_train.columns.astype(str)
processed_df_test.columns = processed_df_test.columns.astype(str)

In [ ]:
random_forest(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])

In [ ]:
logistic_regression(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])

In [ ]:
neural_network(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])

# CSF

In [ ]:
logistic_regression(train_df = CSF_TRAIN, test_df = CSF_TEST, cols = ['CSFABETA', 'CSFPTAU', 'CSFTTAU'])

In [ ]:
random_forest(train_df = CSF_TRAIN, test_df = CSF_TEST, cols = ['CSFABETA', 'CSFPTAU', 'CSFTTAU'])

# UDS_MRI_CSF

## No feature selection

In [ ]:
logistic_regression(train_df = UDS_MRI_CSF_TRAIN, test_df = UDS_MRI_CSF_TEST, cols = UDS_MRI_CSF_TRAIN.columns[:-1])

In [ ]:
random_forest(train_df = UDS_MRI_CSF_TRAIN, test_df = UDS_MRI_CSF_TEST, cols = UDS_MRI_CSF_TRAIN.columns[:-1])

In [ ]:
neural_network(train_df = UDS_MRI_CSF_TRAIN, test_df = UDS_MRI_CSF_TEST, cols = UDS_MRI_CSF_TRAIN.columns[:-1], epochs = 20)

## Feature Selection

### AE + three

In [ ]:
model, scaler = AE(UDS_MRI_CSF_TRAIN, epochs = 50)

In [ ]:
processed_AE_train = through_AE(UDS_MRI_CSF_TRAIN, model, scaler)
processed_AE_test = through_AE(UDS_MRI_CSF_TEST, model, scaler)

In [ ]:
random_forest(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])

In [ ]:
logistic_regression(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])

In [ ]:
neural_network(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])

### PCA + three

In [ ]:
pca_train, pca_test = pca(train_df = UDS_MRI_CSF_TRAIN, test_df = UDS_MRI_CSF_TEST)

In [ ]:
random_forest(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])

In [ ]:
logistic_regression(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:2])

In [ ]:
neural_network(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])

### LASSO + Logistic regression, random forest, neural network

In [ ]:
ze_cols = lasso(df = UDS_MRI_CSF_TRAIN, al = 0.05, thresh = 0.1)
ze_cols

In [ ]:
logistic_regression(train_df = UDS_MRI_CSF_TRAIN, test_df = UDS_MRI_CSF_TEST, cols = ze_cols)

In [ ]:
random_forest(train_df = UDS_MRI_CSF_TRAIN, test_df = UDS_MRI_CSF_TEST, cols = ze_cols)

In [ ]:
neural_network(train_df = UDS_MRI_CSF_TRAIN, test_df = UDS_MRI_CSF_TEST, cols = ze_cols)

### SDNN + Logistic regression, random forest, neural network

In [ ]:
mod, scaler = SDNN(UDS_MRI_CSF_TRAIN, epochs = 15)

In [ ]:
processed_df_train = through_SDNN(UDS_MRI_CSF_TRAIN, mod, scaler)
processed_df_test = through_SDNN(UDS_MRI_CSF_TEST, mod, scaler)

In [ ]:
processed_df_train.columns = processed_df_train.columns.astype(str)
processed_df_test.columns = processed_df_test.columns.astype(str)

In [ ]:
random_forest(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])

In [ ]:
logistic_regression(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])

In [ ]:
neural_network(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1], epochs = 10)

# UDS_MRI

## No feature selection

In [ ]:
logistic_regression(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = UDS_MRI_TRAIN.columns[:-1])

In [ ]:
random_forest(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = UDS_MRI_TRAIN.columns[:-1])

In [ ]:
neural_network(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = UDS_MRI_TRAIN.columns[:-1])

## PCA + three

In [ ]:
pca_train, pca_test = pca(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST)

In [ ]:
random_forest(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])

In [ ]:
logistic_regression(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])

In [ ]:
neural_network(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8], epochs = 15)

## Lasso + Logistic regression, random forest, neural network

In [ ]:
UDS_MRI_TRAIN

In [ ]:
them_cols = lasso(df = UDS_MRI_TRAIN, al = 0.05, thresh = 0.05) # al & thresh .05 worked best
them_cols

In [ ]:
logistic_regression(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = them_cols)

In [ ]:

random_forest(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = lasso(df = UDS_MRI_TRAIN, al = 2 / 100, thresh = 1 / 100))

In [ ]:
tc

In [ ]:
neural_network(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = them_cols)

## SDNN + Logistic regression, random forest, neural network

In [ ]:
mod, scaler = SDNN(UDS_MRI_TRAIN)

In [ ]:
processed_df_train = through_SDNN(UDS_MRI_TRAIN, mod, scaler)
processed_df_test = through_SDNN(UDS_MRI_TEST, mod, scaler)

In [ ]:
processed_df_train.columns = processed_df_train.columns.astype(str)
processed_df_test.columns = processed_df_test.columns.astype(str)

In [ ]:
random_forest(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])

In [ ]:
logistic_regression(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])

In [ ]:
neural_network(train_df = processed_df_train, test_df = processed_df_test, epochs = 3, cols = processed_df_train.columns[:-1])

## AE + the rest

In [ ]:
mod, scaler = AE(UDS_MRI_TRAIN)

In [ ]:
processed_df_train = through_AE(UDS_MRI_TRAIN, mod, scaler)
processed_df_test = through_AE(UDS_MRI_TEST, mod, scaler)
processed_df_train.columns = processed_df_train.columns.astype(str)
processed_df_test.columns = processed_df_test.columns.astype(str)

In [ ]:
logistic_regression(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])

In [ ]:
random_forest(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])

In [ ]:
neural_network(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])

# UDS_MRI FLIPPED - late concatenation

In [ ]:
# This is a bit weird, but due to my data pipeline the merged UDS_MRI doesn't have the same features as UDS and MRI.
# That was to preserve more data in each dataset. So here we are just finding columns in common.

UDS_cols_in_merged = [x for x in UDS_MRI_TRAIN.columns if x not in input_mri_feats]
UDS_cols_use = [x for x in UDS_cols_in_merged if x in UDS_TRAIN.columns]

MRI_cols_in_merged = [x for x in UDS_MRI_TRAIN.columns if x not in UDS_cols_in_merged]
MRI_cols_use = [x for x in MRI_cols_in_merged if x in MRI_TRAIN.columns] + ['NACCUDSD']

## AE + Three


In [ ]:
UDS_mod, UDS_scaler = AE(UDS_TRAIN[UDS_cols_use], epochs = 50)

In [ ]:
MRI_mod, MRI_scaler = AE(MRI_TRAIN[MRI_cols_use], epochs = 50)

In [ ]:
processed_df_UDS_train = through_AE(UDS_MRI_TRAIN[UDS_cols_use], UDS_mod, UDS_scaler)
processed_df_UDS_test = through_AE(UDS_MRI_TEST[UDS_cols_use], UDS_mod, UDS_scaler)

In [ ]:
processed_df_MRI_train = through_AE(UDS_MRI_TRAIN[MRI_cols_use], MRI_mod, MRI_scaler)
processed_df_MRI_test = through_AE(UDS_MRI_TEST[MRI_cols_use], MRI_mod, MRI_scaler)

In [ ]:
merged_ae_train = processed_df_UDS_train[processed_df_UDS_train.columns[:-1]].merge(processed_df_MRI_train[processed_df_MRI_train.columns[:-1]], how = "outer", left_index=True, right_index=True)
merged_ae_train['NACCUDSD'] = processed_df_UDS_train['NACCUDSD']
merged_ae_train

In [ ]:
merged_ae_test = processed_df_UDS_test[processed_df_UDS_test.columns[:-1]].merge(processed_df_MRI_test[processed_df_MRI_test.columns[:-1]], how = "outer", left_index=True, right_index=True)
merged_ae_test['NACCUDSD'] = processed_df_UDS_test['NACCUDSD']
merged_ae_test

In [ ]:
logistic_regression(train_df = merged_ae_train, test_df = merged_ae_test, cols = merged_ae_train.columns[:-1])

In [ ]:
random_forest(train_df = merged_ae_train, test_df = merged_ae_test, cols = merged_ae_train.columns[:-1])

In [ ]:
neural_network(train_df = merged_ae_train, test_df = merged_ae_test, cols = merged_ae_train.columns[:-1], epochs = 50)

## PCA + three

In [ ]:
pca_UDS_train, pca_UDS_test = pca(train_df = UDS_MRI_TRAIN[UDS_cols_use], test_df = UDS_MRI_TEST[UDS_cols_use])

In [ ]:
pca_MRI_train, pca_MRI_test = pca(train_df = UDS_MRI_TRAIN[MRI_cols_use], test_df = UDS_MRI_TEST[MRI_cols_use])

In [ ]:
merged_pca_train = pca_MRI_train[pca_MRI_train.columns[:8]].merge(pca_UDS_train[pca_UDS_train.columns[:8]], how = "outer", left_index=True, right_index=True)
merged_pca_train['NACCUDSD'] = pca_MRI_train['NACCUDSD']
merged_pca_train

In [ ]:
merged_pca_test = pca_MRI_test[pca_MRI_test.columns[:8]].merge(pca_UDS_test[pca_UDS_test.columns[:8]], how = "outer", left_index=True, right_index=True)
merged_pca_test['NACCUDSD'] = pca_MRI_test['NACCUDSD']
merged_pca_test

In [ ]:
random_forest(train_df = merged_pca_train, test_df = merged_pca_test, cols = merged_pca_train.columns[:-1])

In [ ]:
logistic_regression(train_df = merged_pca_train, test_df = merged_pca_test, cols = merged_pca_train.columns[:-1])

In [ ]:
neural_network(train_df = merged_pca_train, test_df = merged_pca_test, cols = merged_pca_train.columns[:-1], epochs = 20)

## LASSO + the three

In [ ]:
UDS_cols = lasso(df = UDS_TRAIN, al = 0.01, thresh = 0.04)
UDS_cols

In [ ]:
MRI_cols = lasso(df = MRI_TRAIN, al = 0.01, thresh = 0.04)
MRI_cols

In [ ]:
logistic_regression(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = UDS_cols+MRI_cols)

In [ ]:
for i in range(10):
  print(random_forest(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = UDS_cols+MRI_cols))

In [ ]:
neural_network(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = UDS_cols+MRI_cols, epochs = 15)

## SDNN + The Three

In [ ]:
UDS_mod, UDS_scaler = SDNN(UDS_TRAIN[UDS_cols_use])

In [ ]:
MRI_mod, MRI_scaler = SDNN(MRI_TRAIN[MRI_cols_use])

In [ ]:
processed_df_UDS_train = through_SDNN(UDS_MRI_TRAIN[UDS_cols_use], UDS_mod, UDS_scaler)
processed_df_UDS_test = through_SDNN(UDS_MRI_TEST[UDS_cols_use], UDS_mod, UDS_scaler)

In [ ]:
processed_df_MRI_train = through_SDNN(UDS_MRI_TRAIN[MRI_cols_use], MRI_mod, MRI_scaler)
processed_df_MRI_test = through_SDNN(UDS_MRI_TEST[MRI_cols_use], MRI_mod , MRI_scaler)

In [ ]:
merged_sdnn_train = processed_df_UDS_train[processed_df_UDS_train.columns[:-1]].merge(processed_df_MRI_train[processed_df_MRI_train.columns[:-1]], how = "outer", left_index=True, right_index=True)
merged_sdnn_train['NACCUDSD'] = processed_df_UDS_train['NACCUDSD']
merged_sdnn_train

In [ ]:
merged_sdnn_test = processed_df_UDS_test[processed_df_UDS_test.columns[:-1]].merge(processed_df_MRI_test[processed_df_MRI_test.columns[:-1]], how = "outer", left_index=True, right_index=True)
merged_sdnn_test['NACCUDSD'] = processed_df_UDS_test['NACCUDSD']
merged_sdnn_test

In [ ]:
merged_sdnn_test

In [ ]:
merged_sdnn_train.columns

In [ ]:
logistic_regression(train_df = merged_sdnn_train, test_df = merged_sdnn_test, cols = merged_sdnn_train.columns[:-1])

In [ ]:
random_forest(train_df = merged_sdnn_train, test_df = merged_sdnn_test, cols = merged_sdnn_train.columns[:-1])

In [ ]:
neural_network(train_df = merged_sdnn_train, test_df = merged_sdnn_test, cols = merged_sdnn_train.columns[:-1])

## PFI

In [ ]:
UDS_cols_use[0]
ground = merged_sdnn_test[merged_sdnn_test.columns[:-1]]
ground

In [281]:
# Function to calculate cosine similarity between two vectors
def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    return dot_product / (norm_A * norm_B)

# Function to calculate cosine similarity row by row and add the result as a new column
def calculate_cosine_similarity_row_by_row(df1, df2):
    cosine_similarities = []
    for index, row in df1.iterrows():
        A = row.values
        B = df2.loc[index].values
        cosine_sim = cosine_similarity(A, B)
        cosine_similarities.append(cosine_sim)
    return cosine_similarities

calculate_cosine_similarity_row_by_row(permute_merged_sdnn_test, ground)

[1.0000001,
 1.0000001,
 1.0,
 1.0,
 1.0,
 1.0000001,
 0.9999999,
 1.0000001,
 1.0,
 1.0000001,
 1.0,
 0.99999994,
 1.0000001,
 1.0,
 1.0000001,
 1.0,
 1.0000001,
 1.0000001,
 0.99999994,
 0.99999994,
 1.0,
 1.0000001,
 1.0000001,
 1.0,
 1.0,
 1.0000001,
 1.0,
 1.0000001,
 0.99999994,
 0.9999999,
 1.0000001,
 1.0000001,
 1.0000001,
 1.0,
 1.0,
 0.99999994,
 1.0,
 1.0,
 1.0000001,
 0.99999994,
 0.99999994,
 1.0000001,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0000001,
 1.0,
 1.0000001,
 1.0000001,
 1.0000001,
 0.99999994,
 1.0,
 1.0,
 0.9999999,
 0.99999994,
 0.9999999,
 1.0000001,
 0.99999994,
 1.0,
 0.99999994,
 1.0,
 1.0000001,
 1.0,
 1.0000001,
 0.9999999,
 1.0000001,
 1.0000001,
 1.0000001,
 1.0000001,
 0.99999994,
 0.9999999,
 1.0000001,
 1.0,
 0.9999999,
 1.0000001,
 1.0,
 1.0,
 0.9999999,
 1.0000001,
 1.0000001,
 1.0000001,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9999999,
 1.0,
 0.9999999,
 1.0,
 1.0000001,
 0.9999999,
 1.0000001,
 0.9999999,
 1.0000001,
 0.99999994,
 0.99999994,
 0.99999994,
 1.0,
 0.9999

In [331]:
feature_importance_scores = []

for col in UDS_cols_use + MRI_cols_use:
    # print(col)
    permute_UDS_MRI_TEST = UDS_MRI_TEST.copy()
    permute_UDS_MRI_TEST[col] = np.random.permutation(permute_UDS_MRI_TEST[col])
    
    processed_df_UDS_test = through_SDNN(permute_UDS_MRI_TEST[UDS_cols_use], UDS_mod, UDS_scaler)
    processed_df_MRI_test = through_SDNN(permute_UDS_MRI_TEST[MRI_cols_use], MRI_mod , MRI_scaler)
    
    permute_merged_sdnn_test = processed_df_UDS_test[processed_df_UDS_test.columns[:-1]].merge(processed_df_MRI_test[processed_df_MRI_test.columns[:-1]], how = "outer", left_index=True, \
                                                                                               right_index=True)
  
    cosine_similarities = calculate_cosine_similarity_row_by_row(permute_merged_sdnn_test, ground)
    average_cosine_similarity = np.mean(cosine_similarities)
    
    feature_importance_score = 1.0 - average_cosine_similarity
    
    feature_importance_scores.append(feature_importance_score)

In [333]:
indices_of_largest = np.argsort(feature_importance_scores)[-10:]
indices_of_largest

array([10,  9,  4,  3,  2, 55, 12,  1,  0, 11], dtype=int64)

In [337]:
thing = np.array(UDS_cols_use + MRI_cols_use)
selected_words = thing[indices_of_largest]
selected_words

array(['PAYATTN', 'EVENTS', 'TAXES', 'BILLS', 'DECSUB', 'DECSUB.1',
       'TRAVEL', 'VEG', 'ANIMALS', 'REMDATES'], dtype='<U8')

# FUSION

## Concatenation + Lasso ✅

In [ ]:
UDS_MRI_CSF

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# Load the dataframe
df = UDS_MRI_CSF.select_dtypes(include=['int', 'float'])

# Create the Lasso model
lasso = Lasso(alpha=0.05)

# Select the features
selector = SelectFromModel(lasso, threshold=0.01)
selector.fit(df.drop("NACCUDSD", axis=1), df["NACCUDSD"])

# Get the selected features
selected_features = selector.get_support()

In [ ]:
# Print the selected features
concat_cols = df.columns[:-1][selected_features].tolist()
concat_cols

## Concatenation + SDDR ✅

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

df = clean_concat.select_dtypes(include=['int', 'float'])

# Separate the input features (explanatory variables) and the response variable
X = df.drop('NACCUDSD', axis=1)  # Input features
y = df['NACCUDSD']  # Response variable

# Perform one-hot encoding on the categorical variable
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_categorical = pd.get_dummies(y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Scale the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(y_categorical.shape[1], activation='softmax'))  # Output layer with softmax activation for categorical variables

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=16, validation_data=(X_test_scaled, y_test))

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print('Accuracy on test set:', accuracy)

In [ ]:
import tensorflow as tf
# Create a new model to get values from the last dense layer before output
last_dense_layer_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)

# Example input row
input_row = X_test_scaled[0]  # Replace with your desired input row

# Obtain values from the last dense layer before output for the input row
output_values = last_dense_layer_model.predict(input_row.reshape(1, -1))

print('Output values:', output_values)

In [ ]:
# successful feature selection?

# CLASSIFICATION

## Random Forest (from concatenation + lasso) ✅

In [ ]:
concat_cols

In [ ]:
y_pred

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the dataset
# dataset = UDS_MRI_CSF[concat_cols+['NACCUDSD']]
X = UDS_MRI_CSF[['TRAILBLI', 'INRACE']]  # Input features # ERROR: y is included
y = UDS_MRI_CSF['NACCUDSD']   # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=50)

# Fit the model to the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


## CNN (from lasso + concat) ✅

In [ ]:
bad_CSF_cols = ['NACCID','CSFLPDY', 'CSFLPYR', 'CSFLPMO', 'Unnamed: 0', 'NACCADC', 'CSFPTDY', 'CSFPTYR', 'CSFPTMO','CSFLPDATE']
clean_CSF_cols = [x for x in clean_CSF.columns.tolist() if x not in bad_CSF_cols]

In [ ]:
col_matrix = [UDS_cols[:9], clean_MRI_cols[:9], clean_CSF_cols[:9]+['CSFABETA']]
col_matrix

In [ ]:
data = UDS_MRI_CSF[col_matrix[0] + col_matrix[1] + col_matrix[2]].to_numpy().reshape(364, 3, 9, 1)

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the dimensions of your input data
input_shape = (3, 9, 1)  # Assuming grayscale images with shape (3, 9)

# Create a small CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))  # Assuming you have a number of output classes

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Assuming you have your features in a numpy array
features = data  # Replace with your actual feature data
labels = UDS['NACCUDSD'].to_numpy()-1# np.random.randint(num_classes, size=(num_samples))  # Replace with your actual labels

# Convert labels to categorical one-hot encoding
labels_one_hot = np.eye(4)[labels]

# Train the model
model.fit(features, labels_one_hot, batch_size=16, epochs=10, validation_split=0.2)


In [ ]:
np.random.randint(44, size=(55))

# OBSOLETE

In [ ]:
for df in [UDS, MRI, CSF, UDS_MRI_CSF]:
    df.mask(df == 8888.888,  np.nan,inplace=True) # MRI ROIs
    df.mask(df == 9999.999,  np.nan,inplace=True)
    df.mask(df == 999.9999,  np.nan,inplace=True)
    df.mask(df == 888.8888,  np.nan,inplace=True)
    df.mask(df == 88.8888,  np.nan,inplace=True)
    df.mask(df == 99.9999,  np.nan,inplace=True)
    df.mask(df == 8.8888,  np.nan,inplace=True)
    df.mask(df == 9.9999,  np.nan,inplace=True)
    df.replace({'EDUC': [99]},  np.nan,inplace=True)
    df.replace({'NACCAPOE': [9]},  np.nan,inplace=True)
    df.rename(columns={'VISITYR': 'year'},inplace=True)



#### UDS_MRI_CSF

In [ ]:
UDS_MRI_CSF

In [ ]:
df = UDS_MRI_CSF

# Count the number of NaNs for each column
nan_counts = df.isnull().sum()

bad_cols = []

# Enumerate the unique counts of NaNs
for nan_count in sorted(set(nan_counts)):
    # Get the columns with the specified number of NaNs
    columns = [column for column, count in nan_counts.items() if count == nan_count]

    if nan_count > 40:
      bad_cols += columns

      # Print the columns
      print(f"Number of NaNs: {nan_count}, columns: {columns}")

In [ ]:
clean_concat = UDS_MRI_CSF.drop(bad_cols, axis=1).dropna()
clean_concat

#### more UDS cleaning

In [ ]:
import numpy as np
import pandas as pd

df = UDS

# Count the number of NaNs for each column
nan_counts = df.isnull().sum()

In [ ]:
bad_cols = []

# Enumerate the unique counts of NaNs
for nan_count in sorted(set(nan_counts)):
    # Get the columns with the specified number of NaNs
    columns = [column for column, count in nan_counts.items() if count == nan_count]

    if nan_count > 20000:
      bad_cols += columns

    # Print the columns
    print(f"Number of NaNs: {nan_count}, columns: {columns}")

In [ ]:
len(bad_cols)

In [ ]:
clean_UDS = UDS.drop(bad_cols, axis=1).dropna()
clean_UDS

#### more MRI cleaning

In [ ]:
import numpy as np
import pandas as pd

df = MRI

# Count the number of NaNs for each column
nan_counts = df.isnull().sum()

bad_cols = []

# Enumerate the unique counts of NaNs
for nan_count in sorted(set(nan_counts)):
    # Get the columns with the specified number of NaNs
    columns = [column for column, count in nan_counts.items() if count == nan_count]

    if nan_count > 5200:
      bad_cols += columns

    # Print the columns
    print(f"Number of NaNs: {nan_count}, columns: {columns}")

In [ ]:
bad_cols += ['MRIYR', 'MRIMO', 'MRIDY', 'MRIDATE']

In [ ]:
clean_MRI = MRI.drop(bad_cols, axis=1).dropna()
clean_MRI

### more CSF cleaning


In [ ]:
import numpy as np
import pandas as pd

df = CSF

# Count the number of NaNs for each column
nan_counts = df.isnull().sum()

bad_cols = []

# Enumerate the unique counts of NaNs
for nan_count in sorted(set(nan_counts)):
    # Get the columns with the specified number of NaNs
    columns = [column for column, count in nan_counts.items() if count == nan_count]

    if nan_count > 1700:
      bad_cols += columns

    # Print the columns
    print(f"Number of NaNs: {nan_count}, columns: {columns}")

In [ ]:
bad_cols += ['CSFLPDY', 'CSFLPYR', 'CSFLPMO', 'Unnamed: 0', 'NACCADC', 'CSFPTDY', 'CSFPTYR', 'CSFPTMO','CSFLPDATE']
clean_CSF = CSF.drop(bad_cols, axis=1).dropna()
clean_CSF

## attempting to fill in naccudsd for MRI and CSF

In [ ]:
mrimo = MRI['MRIMO']
mriday = MRI['MRIDY']
mriyr = MRI['MRIYR']

# Create a new column combining the date components
MRI['MRIDATE'] = pd.to_datetime(mriyr.astype(str) + '-' + mrimo.astype(str) + '-' + mriday.astype(str))
visitmo = UDS['VISITMO']
visitday = UDS['VISITDAY']
visityr = UDS['VISITYR']

# Create a new column combining the date components
UDS['VISITDATE'] = pd.to_datetime(visityr.astype(str) + '-' + visitmo.astype(str) + '-' + visitday.astype(str))

In [ ]:
# Extract the individual columns
# CSF LP = CSF Lumbar Puncture
csfmo = CSF['CSFLPMO']
csfday = CSF['CSFLPDY']
csfyr = CSF['CSFLPYR']

# Create a new column combining the date components
CSF['CSFLPDATE'] = pd.to_datetime(csfyr.astype(str) + '-' + csfmo.astype(str) + '-' + csfday.astype(str))

In [ ]:
def add_diagnosis(datf, date_name):
  # Create an empty column in MRI to store the matched NACCUDSD values
  datf['NACCUDSD'] = ''
  rows_to_delete = []
  # Iterate over each row in the MRI DataFrame
  for index, row in datf.iterrows():
      NACCID = row['NACCID']
      datf_date = row[date_name]

      # Filter UDS DataFrame based on NACCID and VISITDATE conditions
      matching_rows = UDS[(UDS['NACCID'] == NACCID) & (UDS['VISITDATE'] >= datf_date - pd.DateOffset(months=18)) & (UDS['VISITDATE'] <= datf_date)]

      # Check if any matching rows were found
      if matching_rows.empty:
          rows_to_delete.append(index)
          # print(index)
      else:
          # Choose a row from the matching rows (e.g., first row)
          chosen_row = matching_rows.iloc[0]
          NACCUDSD = chosen_row['NACCUDSD']

          # Assign the NACCUDSD value from the chosen row to the corresponding row in MRI
          datf.at[index, 'NACCUDSD'] = NACCUDSD

  datf = datf.drop(rows_to_delete)
  # Display the updated MRI DataFrame
  print(datf)

In [ ]:
add_diagnosis(CSF, 'CSFLPDATE')

## More cleaning

In [ ]:
def deal_with_missing_values_and_add_naccudsd(data, naccudsd, thresh, remove_cols, impute, date_col):

    # High thresh = remove fewer more columns
    # Low thresh = remove more columns

    df = data
    if (naccudsd):
      df = add_diagnosis(data)

    if thresh < 1:
      thresh = thresh * len(data)
      # print(thresh)

    # MASKING NULL VALUES
    df.mask(df == 8888.8888,  np.nan,inplace=True)
    df.mask(df == 8888.888,  np.nan,inplace=True) # MRI ROIs
    df.mask(df == 9999.999,  np.nan,inplace=True)
    df.mask(df == 999.9999,  np.nan,inplace=True)
    df.mask(df == 888.8888,  np.nan,inplace=True)
    df.mask(df == 88.8888,  np.nan,inplace=True)
    df.mask(df == 99.9999,  np.nan,inplace=True)
    df.mask(df == 8.8888,  np.nan,inplace=True)
    df.mask(df == 9.9999,  np.nan,inplace=True)
    df.mask(df == -4,  np.nan,inplace=True)
    df.mask(df == -8,  np.nan,inplace=True)
    df.replace({'EDUC': [99]},  np.nan,inplace=True)
    df.replace({'NACCAPOE': [9]},  np.nan,inplace=True)
    df.rename(columns={'VISITYR': 'year'},inplace=True)

    # Count the number of NaNs for each column
    nan_counts = df.isnull().sum()

    bad_cols = remove_cols

    # print("THE FOLLOWING COLUMNS ARE DROPPED")

    # Enumerate the unique counts of NaNs
    for nan_count in sorted(set(nan_counts)):
        # Get the columns with the specified number of NaNs
        columns = [column for column, count in nan_counts.items() if count == nan_count]


        if nan_count > thresh:
          bad_cols += columns
          # print(f"Number of NaNs: {nan_count}, columns: {columns}")

        # Print the columns

    # print('REMOVING ' + str(len(bad_cols)) + " COLUMNS")

    # remove columns with too many missing data, irrelevant columns
    df = df.drop(bad_cols, axis=1).dropna()

    # if impute:
    # else:
    #   df = df.dropna()

    print("length: " + str(len(df)))
    return df


## Lasso + Concatenation ✅ - what about CSF?

### UDS

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# Load the dataframe
df = clean_UDS.select_dtypes(include=['int', 'float'])

# Create the Lasso model
lasso = Lasso(alpha=0.001)

# Select the features
selector = SelectFromModel(lasso, threshold=0.001)
selector.fit(df.drop("NACCUDSD", axis=1), df["NACCUDSD"])

# Get the selected features
selected_features = selector.get_support()

In [ ]:
# Print the selected features
UDS_cols = df.columns[:-1][selected_features].tolist()
UDS_cols

### MRI

In [ ]:
clean_MRI['NACCUDSD']= clean_MRI['NACCUDSD'].astype(int)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# Load the dataframe
df = clean_MRI.select_dtypes(include=['int', 'float'])

# Create the Lasso model
lasso = Lasso(alpha=0.01)

# Select the features
selector = SelectFromModel(lasso, threshold=0.01)
selector.fit(df.drop("NACCUDSD", axis=1), df["NACCUDSD"])

# Get the selected features
selected_features = selector.get_support()

In [ ]:
# Print the selected features
clean_MRI_cols = df.columns[:-1][selected_features].tolist()
clean_MRI_cols

### CSF

In [ ]:
concat_cols

In [ ]:
clean_CSF = clean_CSF[clean_CSF['NACCUDSD'] != '']
clean_CSF

In [ ]:
clean_CSF['NACCUDSD']= clean_CSF['NACCUDSD'].astype(int)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# Load the dataframe
df = clean_CSF.select_dtypes(include=['int', 'float'])

# Create the Lasso model
lasso = Lasso(alpha=0.001)

# Select the features
selector = SelectFromModel(lasso, threshold=0.001)
selector.fit(df.drop("NACCUDSD", axis=1), df["NACCUDSD"])

# Get the selected features
selected_features = selector.get_support()

In [ ]:
# Print the selected features
clean_CSF_cols = df.columns[:-1][selected_features].tolist()
clean_CSF_cols

In [ ]:
deal_with_missing_values_and_add_naccudsd(data = UDS_MRI_CSF,
                                          naccudsd = False,
                                          thresh = 0,
                                          remove_cols = ['NACCVNUM'],
                                          impute = True,
                                          date_col = 'VISITDATE')

In [ ]:
# for i in range(0,100):
#   print(i)
#   deal_with_missing_values_and_add_naccudsd(data = UDS_MRI_CSF,
#                                           naccudsd = False,
#                                           thresh = i / 100,
#                                           remove_cols = ['NACCVNUM'],
#                                           impute = True,
#                                           date_col = 'VISITDATE')

In [ ]:
UDS_MRI_CSF